In [4]:
from sqlalchemy import create_engine, text, bindparam, Integer
from sqlalchemy.orm import sessionmaker
import pandas as pd

In [19]:
class MeterCaja:
    def __init__(self):
        self.server_solmicro = r'srvsql'
        self.database_solmicro = 'SolmicroERP6_PruebasSub'
        self.username_solmicro = 'sa'
        self.password_solmicro = 'Altai2021'
        self.engine = create_engine(f'mssql+pyodbc://{self.username_solmicro}:{self.password_solmicro}@{self.server_solmicro}/{self.database_solmicro}?driver=SQL+Server')
        self.connection = None
        self.tables_info = []
        self.ruta_archivo = r'final.xlsx'
        self.hoja_excel = 'Hoja2'
        self.listArticulos = [443000021633, 442295636741, 443000021629, 443000022021, 443000022026, 443000022027, 443000021525, 443000021559, 443000021561, 443000021562, 443000021563,443000021565, 443000022076, 442296340551, 443000021582, 443000021583, 443000021584, 443000021585, 443000021627, 443000022005, 443000021626, 443000021625, 443000016122, 443000021628]

    def CargarExcel(self):
        try:
            datos_excel = pd.read_excel(self.ruta_archivo, sheet_name=self.hoja_excel)
            nombre_columna = 'ID'
            columna_especifica = datos_excel['ID']
            art_ids = set(item for item in columna_especifica)
            return art_ids
        except Exception as e:
            print("Error Excel: ", e)

    def Open_Conn_Solmicro(self):
        try:
            self.connection = self.engine.connect()
            return self.connection
        except Exception as e:
            print("Error opening connection: ", e)

    def CargarExcel(self):
        try:
            datos_excel = pd.read_excel(self.ruta_archivo, sheet_name=self.hoja_excel)
            nombre_columna = 'ID'  
            columna_especifica = datos_excel['ID'] 
            art_ids = set(item for item in columna_especifica) 
            return art_ids
        except Exception as e:
            print("Error Excel: ",e)
    
    def getDatosRuta(self, datos_excel):
        resultados = []
        try:
            conn = self.Open_Conn_Solmicro()
            if conn:
                for id in datos_excel:
                    query = text(f"SELECT TOP(1) IDArticulo, Secuencia FROM tbRuta WHERE (IDArticulo = '{id}') ORDER BY Secuencia")
                    result = conn.execute(query).fetchone()
                    if result:
                        # print(result)
                        resultados.append(result)
                conn.close()
            return resultados
        except Exception as e:
            print("Error en la consulta:", e)
            return resultados
        
    def insertSecc(self,List_id, autonumeric):
        idx = 0
        conn = None
        try:
            conn = self.Open_Conn_Solmicro()
            if conn:
                for id, sec in List_id:
                    idx = idx + 1
                    if(sec == 5):
                        stringInsert = text(f"insert into tbRuta ([IDRutaOp], [IDArticulo], [IDRuta], [Secuencia], [TipoOperacion], [IDOperacion], [DescOperacion], [Critica], [IDCentro], [FactorHombre], [IDUdProduccion], [UdTiempo],[FactorProduccion], [ControlProduccion], [UdTiempoPrep], [UdTiempoEjec], [FechaCreacionAudi], [FechaModificacionAudi],[UsuarioAudi], [UsuarioCreacionAudi]) values ({autonumeric[idx]}, N'{id}', N'01', 6, 0, N'6', N'METER CAJA EMBALAJE + INDICAR MAXIMO PIEZAS CAJA Y  PALET S/ LISTA', 0, N'1200', 1, N'u.', 2, 1, 1, 2, 2, '20231123 10:36:58.865', '20231123 10:36:58.865', N'favram\a.obregon', N'favram\a.obregon')")
                        conn.execute(stringInsert)
                        conn.commit()
                    else:
                        stringInsert = text(f"insert into tbRuta ([IDRutaOp], [IDArticulo], [IDRuta], [Secuencia], [TipoOperacion], [IDOperacion], [DescOperacion], [Critica], [IDCentro], [FactorHombre], [IDUdProduccion], [UdTiempo],[FactorProduccion], [ControlProduccion], [UdTiempoPrep], [UdTiempoEjec], [FechaCreacionAudi], [FechaModificacionAudi],[UsuarioAudi], [UsuarioCreacionAudi]) values ({autonumeric[idx]}, N'{id}', N'01', 5, 0, N'5', N'METER CAJA EMBALAJE + INDICAR MAXIMO PIEZAS CAJA Y  PALET S/ LISTA', 0, N'1200', 1, N'u.', 2, 1, 1, 2, 2, '20231123 10:36:58.865', '20231123 10:36:58.865', N'favram\a.obregon', N'favram\a.obregon')")
                        conn.execute(stringInsert)
                        conn.commit()
        except Exception as e:
            print("Error ejecutando el procedimiento almacenado:", e)
            return None
        finally:
            if conn:
                conn.close()
    
    def insertDocuments(self,articulos,autonumeric):
        idx = 0
        conn = None
        try:
            conn = self.Open_Conn_Solmicro()
            if conn:
                for id in articulos:
                    print(id)
                    stringInsert = text(f"insert into tbDcmDocumentoEntidad ([IDDocEntidad], [Entidad], [IdDocumento], [Campo1], [Valor1], [FechaCreacionAudi], [FechaModificacionAudi], [UsuarioAudi], [UsuarioCreacionAudi]) values ({autonumeric[idx]}, N'Articulo', 19606213, N'IDArticulo', N'{id}', '20231124 08:09:58.424', '20231124 08:09:58.424', N'favram\a.obregon', N'favram\a.obregon')")
                    conn.execute(stringInsert)
                    conn.commit()
                    idx = idx + 1
        except Exception as e:
            print("Error de insercion", e)
            return None
        finally:
            if conn:
                conn.close()
        


    def RunProcedure(self,datosExcel):
        conn = None
        try:
            conn = self.Open_Conn_Solmicro()
            if conn:
                for dato in datosExcel:
                    query = text("EXEC xAutonumericValue;")
                    conn.execute(query)
                    conn.commit()
        except Exception as e:
            print("Error ejecutando el procedimiento almacenado:", e)
            return None
        finally:
            if conn:
                conn.close()

    def GetAutonumericValue(self):
        conn = None
        try:
            conn = self.Open_Conn_Solmicro()
            if conn:
                query = text("SELECT autonumber FROM  ale_autoNumber")
                result = conn.execute(query).fetchall()
                final_result = [ value for row in result for value in row ]
                return final_result
        except Exception as e:
            print("Error ejecutando el procedimiento almacenado:", e)
            return None
        finally:
            if conn:
                conn.close()

In [20]:

micaja = MeterCaja()
# datosExcel = micaja.CargarExcel()

# autonumeric = micaja.GetAutonumericValue()

In [7]:
listSet = list(set(micaja.listArticulos) - set([443000022021, 443000016122]) - set([443000021633, 442295636741, 443000021629]))

In [8]:
micaja.RunProcedure(datosExcel=listSet)

In [9]:
autonumeric = micaja.GetAutonumericValue()

In [21]:
list_id = micaja.getDatosRuta(datos_excel=listSet)

In [22]:
list_id

[('443000021565', 1),
 ('442296340551', 1),
 ('443000022026', 1),
 ('443000022027', 1),
 ('443000021582', 1),
 ('443000021583', 1),
 ('443000021584', 1),
 ('443000021585', 1),
 ('443000022076', 1),
 ('443000021525', 1),
 ('443000021563', 1),
 ('443000021561', 1),
 ('443000021562', 1),
 ('443000022005', 1),
 ('443000021559', 1),
 ('443000021625', 1),
 ('443000021626', 1),
 ('443000021627', 1),
 ('443000021628', 1)]

In [23]:
micaja.insertSecc(List_id=list_id,autonumeric=autonumeric)

Error ejecutando el procedimiento almacenado: list index out of range


In [78]:
# micaja.insertSecc(datosRutas,autonumeric=autonumeric)